In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, accuracy_score


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
df=pd.read_csv('looks like A.csv')
df.head()

,Date List Produced,Record Expiration Date,Company Name,AMR_Fuzzy,Verified Record,Executive First Name,Executive Last Name,Executive Title,Role,Department,...,Franchise6_1,Affiliated Records,Affiliated Locations,NAICS,Government Contractor,ECommerce,amr,revenue_bucket,CLASS,cust_id
0,00:00.0,00:00.0,Bridgewater State Hospital,C,Yes,Kenneth W,Nelson,Administrator,Manager,General/Operations,...,NaN,0,0,62211002,NaN,NaN,683.0,A,>$500,6124981.0
1,00:00.0,00:00.0,Woodbridge Fire Dept,C,Yes,Sean C,Rowland,President,Chief Executive Officer/President/Executive Di...,General/Operations,...,NaN,0,0,92216003,NaN,NaN,583.0,A,>$500,1443278.0
2,00:00.0,00:00.0,Otto Herrmann Inc,C,Yes,Richard,Stier,President,Chief Executive Officer/President/Executive Di...,General/Operations,...,NaN,0,0,44413005,Y,Y,935.0,A,>$500,2893821.0
3,00:00.0,00:00.0,Achievement First Rhode Island,C,Yes,Libby,Miller,Principal,Manager,None,...,NaN,0,0,99999005,Y,NaN,546.0,A,>$500,7774182.0
4,00:00.0,00:00.0,Beer King Inc,C,Yes,Johnny,Ling,President,Chief Executive Officer/President/Executive Di...,General/Operations,...,NaN,0,0,44531001,NaN,NaN,1121.0,A,>$500,5527152.0


In [3]:
data=df.copy()

In [4]:
x=(data.isnull().sum()/len(df.index))*100
a=x[x>40.0]
b=a.index
b

Index(['Professional Title', 'Neighborhood', 'Corporate Employee Size Actual',
       'Corporate Employee Size Range', 'Corporate Sales Volume Range',
       'Primary SIC Ad Size', 'Ad Size 1', 'NAICS 1', 'NAICS 1 Description',
       'Primary Franchise/Specialty',
       ...
       'Franchise3_0', 'Franchise3_1', 'Franchise4_0', 'Franchise4_1',
       'Franchise5_0', 'Franchise5_1', 'Franchise6_0', 'Franchise6_1',
       'Government Contractor', 'ECommerce'],
      dtype='object', length=109)

In [5]:
data1=data.drop(b,axis=1)

#### Dropping all those columns that have missing data more than 40%

## EDA

In [6]:
(data1.isnull().sum()/len(data1.index))*100

Date List Produced        0.000000
Record Expiration Date    0.000000
Company Name              0.000000
AMR_Fuzzy                 0.000000
Verified Record           0.000000
                            ...   
NAICS                     0.000000
amr                       0.554318
revenue_bucket            0.554318
CLASS                     0.554318
cust_id                   0.554318
Length: 76, dtype: float64

In [7]:
data_new=data1[data1['revenue_bucket'].notnull()]

#### Considering all those rows in which 'revenue_bucket' is not null as 'revenue_bucket' is our target variable

In [8]:
#data_new.columns

In [9]:
data_new[['Mailing Address',
       'Mailing City', 'Mailing State', 'Mailing Zip Code', 'Mailing Zip 4',
       'Mailing Carrier Route', 'Mailing Delivery Point Barcode',
       'Location Address', 'Location City', 'Location State',
       'Location ZIP Code', 'Location ZIP+4',
       'Location Address Delivery Point Barcode']].head()

,Mailing Address,Mailing City,Mailing State,Mailing Zip Code,Mailing Zip 4,Mailing Carrier Route,Mailing Delivery Point Barcode,Location Address,Location City,Location State,Location ZIP Code,Location ZIP+4,Location Address Delivery Point Barcode
0,20 Administration Rd,Bridgewater,MA,2324,3201.0,R099,201.0,20 Administration Rd,Bridgewater,MA,2324,3201.0,201.0
1,PO Box F,Woodbridge,CT,6525,NaN,R011,NaN,100 Center Rd,Woodbridge,CT,6525,NaN,NaN
2,6729 Myrtle Ave # 1,Glendale,NY,11385,7085.0,C037,291.0,6729 Myrtle Ave # 1,Glendale,NY,11385,7085.0,291.0
3,370 Hartford Ave,Providence,RI,2909,5104.0,C029,703.0,370 Hartford Ave,Providence,RI,2909,5104.0,703.0
4,2102 Utica Ave # 7,Brooklyn,NY,11234,3834.0,C075,74.0,2102 Utica Ave # 7,Brooklyn,NY,11234,3834.0,74.0


In [10]:
data_new.shape

(162897, 76)

In [11]:
data_new[data_new['Mailing City']!=data_new['Location City']].count()

Date List Produced        1977
Record Expiration Date    1977
Company Name              1977
AMR_Fuzzy                 1977
Verified Record           1977
                          ... 
NAICS                     1977
amr                       1977
revenue_bucket            1977
CLASS                     1977
cust_id                   1977
Length: 76, dtype: int64

#### In almost 99% of the data points, mailing address is same as Location address so we'll keep only one column

In [12]:
data_new=data_new.drop(['Mailing Address',
       'Mailing City', 'Mailing State', 'Mailing Zip Code', 'Mailing Zip 4',
       'Mailing Carrier Route', 'Mailing Delivery Point Barcode',
       'Location Address',
       'Location ZIP Code', 'Location ZIP+4',
       'Location Address Delivery Point Barcode',
       'Location Address Carrier Route'],axis=1)

In [13]:
data_new['Date List Produced'].unique()

array(['00:00.0'], dtype=object)

In [14]:
data_new['Record Expiration Date'].unique()

array(['00:00.0'], dtype=object)

In [15]:
data_new=data_new.drop(['Date List Produced', 'Record Expiration Date'],axis=1)

#### Dropping 'Date List Produced' and 'Record Expiration Date' columns as they are not giving us any relevent information

In [16]:
#data_new.columns

In [17]:
data_new['Verified Record'].unique()

array(['Yes'], dtype=object)

In [18]:
data_new=data_new.drop('Verified Record',axis=1)

#### will not consider 'Verified Record' column as value for each data point is same(it is not giving us any additional information)

In [19]:
data_new[['Location Employee Size Actual',
       'Location Employee Size Range', 'Location Sales Volume Actual',
       'Location Sales Volume Range', 'Corporate Sales Volume Actual']].head()

,Location Employee Size Actual,Location Employee Size Range,Location Sales Volume Actual,Location Sales Volume Range,Corporate Sales Volume Actual
0,500.0,500 to 999,74549000,$50-100 Million,0.0
1,50.0,50 to 99,0,NaN,0.0
2,2.0,1 to 4,439000,"Less Than $500,000",0.0
3,4.0,1 to 4,0,NaN,0.0
4,3.0,1 to 4,1035000,$1-2.5 Million,0.0


In [20]:
data_new=data_new.drop(['Location Employee Size Range','Location Sales Volume Range'],axis=1)

#### Will consider only actual values and not the range values

In [21]:
data_new.columns

Index(['Company Name', 'AMR_Fuzzy', 'Executive First Name',
       'Executive Last Name', 'Executive Title', 'Role', 'Department',
       'Functional Area', 'Executive Gender', 'Location City',
       'Location State', 'County Description', 'Metro Area', 'Latitude',
       'Longitude', 'Location Employee Size Actual',
       'Location Sales Volume Actual', 'Corporate Sales Volume Actual',
       'Primary SIC Code', 'Primary SIC Code Description',
       'Primary SIC Year Appeared', 'SIC Code 1', 'SIC Code Description 1',
       'Year Appeared 1', 'Primary NAICS Description', 'Holding Status',
       'Home Business', 'Fortune 1000 Ranking', 'Year Established',
       'Square Footage', 'Infogroup ID', 'Parent Infogroup ID', 'First Name 0',
       'Last Name 0', 'Executive Title 0', 'Executive Gender 0',
       'Accounting Expenses', 'Advertising Expense', 'Computer Expenses',
       'Contract Labor Expenses', 'Insurance Expenses', 'Legal Expenses',
       'Management/Administration Expen

In [22]:
data_new[['Primary SIC Code', 'Primary SIC Code Description',
       'Primary SIC Year Appeared', 'SIC Code 1', 'SIC Code Description 1',
       'Year Appeared 1', 'Primary NAICS Description', 'Holding Status']].head()

,Primary SIC Code,Primary SIC Code Description,Primary SIC Year Appeared,SIC Code 1,SIC Code Description 1,Year Appeared 1,Primary NAICS Description,Holding Status
0,8062-02,Hospitals,2010,8062-02,Hospitals,2010,General Medical & Surgical Hospitals,0
1,9224-04,Fire Departments,1995,9224-04,Fire Departments,1995,Fire Protection,0
2,5251-04,Hardware-Retail,1984,5251-04,Hardware-Retail,1984,Hardware Stores,0
3,9999-66,Federal Government Contractors,2015,9999-66,Federal Government Contractors,2015,Unclassified Establishments,0
4,5921-04,Beer & Ale-Retail,2011,5921-04,Beer & Ale-Retail,2011,"Beer, Wine & Liquor Stores",0


In [23]:
data_new=data_new.drop(['SIC Code 1','SIC Code Description 1','Year Appeared 1',\
                        'Primary SIC Year Appeared'],axis=1)

#### Keeping only primary code and description

In [24]:
data_new['Holding Status'].value_counts()

0    153480
2      9195
1       222
Name: Holding Status, dtype: int64

In [25]:
data_new[['Home Business', 'Fortune 1000 Ranking', 'Year Established',
       'Square Footage', 'Infogroup ID', 'Parent Infogroup ID']].head(2)

,Home Business,Fortune 1000 Ranking,Year Established,Square Footage,Infogroup ID,Parent Infogroup ID
0,No,0,0,"100,000+",362806978,854085644
1,No,0,1929,"100,000+",450418785,0


In [26]:
data_new['Home Business'].value_counts()

No     160218
Yes      2679
Name: Home Business, dtype: int64

In [27]:
data_new=data_new.drop(['Infogroup ID', 'Parent Infogroup ID'],axis=1)

In [28]:
data_new['Fortune 1000 Ranking'].value_counts()

0      162848
415         1
577         1
960         1
63          1
62          1
61          1
310         1
820         1
559         1
173         1
44          1
542         1
378         1
30          1
923         1
404         1
19          1
18          1
772         1
258         1
769         1
640         1
512         1
195         1
254         1
71          1
583         1
248         1
247         1
630         1
501         1
242         1
114         1
369         1
106         1
231         1
228         1
221         1
860         1
217         1
343         1
470         1
341         1
84          1
973         1
972         1
714         1
202         1
581         1
Name: Fortune 1000 Ranking, dtype: int64

In [29]:
data_new['Fortune 1000 Ranking']=data_new['Fortune 1000 Ranking'].map(lambda x: 0 if x==0 else 1)

In [30]:
data_new['Fortune 1000 Ranking'].value_counts()

0    162848
1        49
Name: Fortune 1000 Ranking, dtype: int64

#### Keeping value 0 for companies that did not feature in  Fortune 1000 Ranking and 1 for companies that featured

In [31]:
data_new.columns

Index(['Company Name', 'AMR_Fuzzy', 'Executive First Name',
       'Executive Last Name', 'Executive Title', 'Role', 'Department',
       'Functional Area', 'Executive Gender', 'Location City',
       'Location State', 'County Description', 'Metro Area', 'Latitude',
       'Longitude', 'Location Employee Size Actual',
       'Location Sales Volume Actual', 'Corporate Sales Volume Actual',
       'Primary SIC Code', 'Primary SIC Code Description',
       'Primary NAICS Description', 'Holding Status', 'Home Business',
       'Fortune 1000 Ranking', 'Year Established', 'Square Footage',
       'First Name 0', 'Last Name 0', 'Executive Title 0',
       'Executive Gender 0', 'Accounting Expenses', 'Advertising Expense',
       'Computer Expenses', 'Contract Labor Expenses', 'Insurance Expenses',
       'Legal Expenses', 'Management/Administration Expenses',
       'Office Supplies Expenses', 'Package/Container Expenses',
       'Payroll and Benefits Expenses', 'Purchase Print Expenses',
   

In [32]:
data_new[['Company Name', 'AMR_Fuzzy', 'Executive First Name',
       'Executive Last Name', 'Executive Title', 'Role', 'Department',
       'Functional Area', 'Executive Gender']].head()

,Company Name,AMR_Fuzzy,Executive First Name,Executive Last Name,Executive Title,Role,Department,Functional Area,Executive Gender
0,Bridgewater State Hospital,C,Kenneth W,Nelson,Administrator,Manager,General/Operations,General/Operations - Operations/Administrator,Male
1,Woodbridge Fire Dept,C,Sean C,Rowland,President,Chief Executive Officer/President/Executive Di...,General/Operations,General/Operations - Operations Administration,Male
2,Otto Herrmann Inc,C,Richard,Stier,President,Chief Executive Officer/President/Executive Di...,General/Operations,General/Operations - Operations Administration,Male
3,Achievement First Rhode Island,C,Libby,Miller,Principal,Manager,None,None - General Operations,Female
4,Beer King Inc,C,Johnny,Ling,President,Chief Executive Officer/President/Executive Di...,General/Operations,General/Operations - Operations Administration,Male


In [33]:
data_new['AMR_Fuzzy'].value_counts()

C    154656
A      4881
B      3360
Name: AMR_Fuzzy, dtype: int64

In [34]:
data_new=data_new.drop(['Company Name', 'Executive First Name',
       'Executive Last Name', 'Executive Title', 'Role', 'Department',
       'Functional Area', 'Executive Gender','First Name 0', 'Last Name 0', 'Executive Title 0',
       'Executive Gender 0'],axis=1)

#### Dropping above columns as they don't give any additional information regarding the target variable

In [35]:
data_new[['Tags',
       'Affiliated Records', 'Affiliated Locations', 'NAICS', 'amr',
       'revenue_bucket', 'CLASS', 'cust_id']]

,Tags,Affiliated Records,Affiliated Locations,NAICS,amr,revenue_bucket,CLASS,cust_id
0,Customer / Analytics,0,0,62211002,683.0,A,>$500,6124981.0
1,Customer / Analytics,0,0,92216003,583.0,A,>$500,1443278.0
2,Customer / Analytics,0,0,44413005,935.0,A,>$500,2893821.0
3,Customer / Analytics,0,0,99999005,546.0,A,>$500,7774182.0
4,Customer / Analytics,0,0,44531001,1121.0,A,>$500,5527152.0
...,...,...,...,...,...,...,...,...
162892,Customer / Analytics,0,0,45311001,10.0,G,$(0-20),3590472.0
162893,Customer / Analytics,0,0,54132004,19.0,G,$(0-20),5048857.0
162894,Customer / Analytics,0,0,62199921,17.0,G,$(0-20),6157670.0
162895,Customer / Analytics,0,0,62121003,8.0,G,$(0-20),1100882.0


In [36]:
data_new['Tags'].value_counts()

Customer / Analytics    162897
Name: Tags, dtype: int64

In [37]:
data_new['Affiliated Records'].value_counts()

0       160783
1          373
2          325
3          205
4          143
         ...  
7218         1
50           1
306          1
85           1
184          1
Name: Affiliated Records, Length: 205, dtype: int64

In [38]:
data_new=data_new.drop(['CLASS', 'cust_id','Tags'],axis=1)

In [39]:
data_new.columns

Index(['AMR_Fuzzy', 'Location City', 'Location State', 'County Description',
       'Metro Area', 'Latitude', 'Longitude', 'Location Employee Size Actual',
       'Location Sales Volume Actual', 'Corporate Sales Volume Actual',
       'Primary SIC Code', 'Primary SIC Code Description',
       'Primary NAICS Description', 'Holding Status', 'Home Business',
       'Fortune 1000 Ranking', 'Year Established', 'Square Footage',
       'Accounting Expenses', 'Advertising Expense', 'Computer Expenses',
       'Contract Labor Expenses', 'Insurance Expenses', 'Legal Expenses',
       'Management/Administration Expenses', 'Office Supplies Expenses',
       'Package/Container Expenses', 'Payroll and Benefits Expenses',
       'Purchase Print Expenses', 'Rent Expenses', 'Telcom Expenses',
       'Transportation Expenses', 'Utilities Expense', 'Affiliated Records',
       'Affiliated Locations', 'NAICS', 'amr', 'revenue_bucket'],
      dtype='object')

In [40]:
data_new.isnull().sum()

AMR_Fuzzy                                 0
Location City                             0
Location State                            0
County Description                        0
Metro Area                              315
Latitude                                  0
Longitude                                 0
Location Employee Size Actual          4380
Location Sales Volume Actual              0
Corporate Sales Volume Actual             0
Primary SIC Code                          0
Primary SIC Code Description              0
Primary NAICS Description                 0
Holding Status                            0
Home Business                             0
Fortune 1000 Ranking                      0
Year Established                          0
Square Footage                         8518
Accounting Expenses                   38570
Advertising Expense                   32084
Computer Expenses                     32084
Contract Labor Expenses               32084
Insurance Expenses              

In [41]:
data_new=data_new.drop('Metro Area',axis=1)

## Filling the null values and changing categorical features to numerical features

In [42]:
ls=['Accounting Expenses', 'Advertising Expense', 'Computer Expenses',
       'Contract Labor Expenses', 'Insurance Expenses', 'Legal Expenses',
       'Management/Administration Expenses', 'Office Supplies Expenses',
       'Package/Container Expenses', 'Payroll and Benefits Expenses',
       'Purchase Print Expenses', 'Rent Expenses', 'Telcom Expenses',
       'Transportation Expenses', 'Utilities Expense']
for i in ls:
    lis = [j for j in data_new[i].value_counts().index]
    data_new[i].fillna(data_new[i].mode()[0],inplace=True)
    data_new[i] = data_new[i].map(lambda x: lis.index(x)+1)

In [43]:
data_new[['Accounting Expenses', 'Advertising Expense', 'Computer Expenses',
       'Contract Labor Expenses', 'Insurance Expenses', 'Legal Expenses',
       'Management/Administration Expenses', 'Office Supplies Expenses',
       'Package/Container Expenses', 'Payroll and Benefits Expenses',
       'Purchase Print Expenses', 'Rent Expenses', 'Telcom Expenses',
       'Transportation Expenses', 'Utilities Expense']].head()

,Accounting Expenses,Advertising Expense,Computer Expenses,Contract Labor Expenses,Insurance Expenses,Legal Expenses,Management/Administration Expenses,Office Supplies Expenses,Package/Container Expenses,Payroll and Benefits Expenses,Purchase Print Expenses,Rent Expenses,Telcom Expenses,Transportation Expenses,Utilities Expense
0,7,6,7,6,6,7,6,7,2,7,5,6,7,7,5
1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,3,2,5,3,1,1,2,1,1,2,1,1,2,2,2
3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,4,2,2,3,3,1,4,1,2,2,3,1,2,2,2


In [44]:
data_new.isnull().sum()

AMR_Fuzzy                                0
Location City                            0
Location State                           0
County Description                       0
Latitude                                 0
Longitude                                0
Location Employee Size Actual         4380
Location Sales Volume Actual             0
Corporate Sales Volume Actual            0
Primary SIC Code                         0
Primary SIC Code Description             0
Primary NAICS Description                0
Holding Status                           0
Home Business                            0
Fortune 1000 Ranking                     0
Year Established                         0
Square Footage                        8518
Accounting Expenses                      0
Advertising Expense                      0
Computer Expenses                        0
Contract Labor Expenses                  0
Insurance Expenses                       0
Legal Expenses                           0
Management/

In [45]:
data_new['Square Footage'].fillna(data_new['Square Footage'].mode()[0],inplace=True)

In [46]:
lis = [j for j in data_new['Square Footage'].value_counts().index]
data_new['Square Footage'] = data_new['Square Footage'].map(lambda x: lis.index(x)+1)

In [47]:
data_new.groupby('revenue_bucket')['Location Employee Size Actual'].mean()

revenue_bucket
A    88.242356
B    90.975357
C    57.890470
D    50.984190
E    38.118519
F    23.064909
G    14.574726
Name: Location Employee Size Actual, dtype: float64

In [48]:
data_new['Location Employee Size Actual'].mean()

20.186017903442533

In [49]:
data_new.rename(columns={'Location Employee Size Actual':'Location_Employee_Size_Actual'},inplace=True)

In [50]:
data_new.columns

Index(['AMR_Fuzzy', 'Location City', 'Location State', 'County Description',
       'Latitude', 'Longitude', 'Location_Employee_Size_Actual',
       'Location Sales Volume Actual', 'Corporate Sales Volume Actual',
       'Primary SIC Code', 'Primary SIC Code Description',
       'Primary NAICS Description', 'Holding Status', 'Home Business',
       'Fortune 1000 Ranking', 'Year Established', 'Square Footage',
       'Accounting Expenses', 'Advertising Expense', 'Computer Expenses',
       'Contract Labor Expenses', 'Insurance Expenses', 'Legal Expenses',
       'Management/Administration Expenses', 'Office Supplies Expenses',
       'Package/Container Expenses', 'Payroll and Benefits Expenses',
       'Purchase Print Expenses', 'Rent Expenses', 'Telcom Expenses',
       'Transportation Expenses', 'Utilities Expense', 'Affiliated Records',
       'Affiliated Locations', 'NAICS', 'amr', 'revenue_bucket'],
      dtype='object')

In [51]:
data_new['Location_Employee_Size_Actual'].isnull().sum()

4380

In [52]:
def missing_value(cols):
    revenue_bucket=cols[0]
    Location_Employee_Size_Actual=cols[1]
    if pd.isnull(Location_Employee_Size_Actual):
        if revenue_bucket == "A":
            return 88.242
        elif revenue_bucket == "B":
            return 90.975
        elif revenue_bucket == "C":
            return 57.890
        elif revenue_bucket == "D":
            return 50.984
        elif revenue_bucket == "E":
            return 38.119
        elif revenue_bucket == "F":
            return 23.065
        else:
            return 14.575
    else:
        pass
    return Location_Employee_Size_Actual


In [54]:
data_new['Location_Employee_Size_Actual'].isnull().sum()

4380

In [55]:
data_new["Location_Employee_Size_Actual"] = data_new[['revenue_bucket','Location_Employee_Size_Actual']].apply(missing_value, axis = 1)

In [56]:
data_new.dtypes

AMR_Fuzzy                              object
Location City                          object
Location State                         object
County Description                     object
Latitude                              float64
Longitude                             float64
Location_Employee_Size_Actual         float64
Location Sales Volume Actual            int64
Corporate Sales Volume Actual         float64
Primary SIC Code                       object
Primary SIC Code Description           object
Primary NAICS Description              object
Holding Status                          int64
Home Business                          object
Fortune 1000 Ranking                    int64
Year Established                        int64
Square Footage                          int64
Accounting Expenses                     int64
Advertising Expense                     int64
Computer Expenses                       int64
Contract Labor Expenses                 int64
Insurance Expenses                

In [57]:
data_new.columns

Index(['AMR_Fuzzy', 'Location City', 'Location State', 'County Description',
       'Latitude', 'Longitude', 'Location_Employee_Size_Actual',
       'Location Sales Volume Actual', 'Corporate Sales Volume Actual',
       'Primary SIC Code', 'Primary SIC Code Description',
       'Primary NAICS Description', 'Holding Status', 'Home Business',
       'Fortune 1000 Ranking', 'Year Established', 'Square Footage',
       'Accounting Expenses', 'Advertising Expense', 'Computer Expenses',
       'Contract Labor Expenses', 'Insurance Expenses', 'Legal Expenses',
       'Management/Administration Expenses', 'Office Supplies Expenses',
       'Package/Container Expenses', 'Payroll and Benefits Expenses',
       'Purchase Print Expenses', 'Rent Expenses', 'Telcom Expenses',
       'Transportation Expenses', 'Utilities Expense', 'Affiliated Records',
       'Affiliated Locations', 'NAICS', 'amr', 'revenue_bucket'],
      dtype='object')

### Transforming categorical columns to numerical columns using label encoder

In [58]:
class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [59]:
from sklearn.preprocessing import LabelEncoder
data_new=MultiColumnLabelEncoder(columns = ['Location City', 'Location State', 'County Description',\
                                  'Primary SIC Code', 'Primary SIC Code Description',
       'Primary NAICS Description',]).fit_transform(data_new)

In [60]:
data_new


,AMR_Fuzzy,Location City,Location State,County Description,Latitude,Longitude,Location_Employee_Size_Actual,Location Sales Volume Actual,Corporate Sales Volume Actual,Primary SIC Code,...,Purchase Print Expenses,Rent Expenses,Telcom Expenses,Transportation Expenses,Utilities Expense,Affiliated Records,Affiliated Locations,NAICS,amr,revenue_bucket
0,C,185,18,194,41.944249,-70.952391,500.0,74549000,0.0,4240,...,5,6,7,7,5,0,0,62211002,683.0,A
1,C,1845,6,172,41.340796,-73.015005,50.0,0,0.0,4904,...,1,1,1,1,1,0,0,92216003,583.0,A
2,C,578,31,201,40.701625,-73.885304,2.0,439000,0.0,2472,...,1,1,2,2,2,0,0,44413005,935.0,A
3,C,1310,36,199,41.817417,-71.456242,4.0,0,0.0,4968,...,1,1,1,1,1,0,0,99999005,546.0,A
4,C,200,31,129,40.619880,-73.927260,3.0,1035000,0.0,2877,...,3,1,2,2,2,0,0,44531001,1121.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162892,C,917,31,168,40.676757,-73.460447,10.0,1036000,0.0,3036,...,4,4,3,4,3,0,0,45311001,10.0,G
162893,C,1390,31,168,40.799930,-73.658951,4.0,505000,0.0,51,...,2,2,3,2,2,0,0,54132004,19.0,G
162894,C,1368,31,201,40.582394,-73.837646,12.0,1888000,0.0,4285,...,4,3,1,5,3,0,0,62199921,17.0,G
162895,C,95,31,168,40.907127,-73.552648,15.0,2080000,0.0,4163,...,4,5,1,4,3,0,0,62121003,8.0,G


## Building the model

In [74]:
x=data_new.drop('revenue_bucket',axis=1)
y=data_new['revenue_bucket']

In [75]:
x=pd.get_dummies(x,drop_first=True)

In [76]:
x.head()

,Location City,Location State,County Description,Latitude,Longitude,Location_Employee_Size_Actual,Location Sales Volume Actual,Corporate Sales Volume Actual,Primary SIC Code,Primary SIC Code Description,...,Telcom Expenses,Transportation Expenses,Utilities Expense,Affiliated Records,Affiliated Locations,NAICS,amr,AMR_Fuzzy_B,AMR_Fuzzy_C,Home Business_Yes
0,185,18,194,41.944249,-70.952391,500.0,74549000,0.0,4240,2286,...,7,7,5,0,0,62211002,683.0,0,1,0
1,1845,6,172,41.340796,-73.015005,50.0,0,0.0,4904,1766,...,1,1,1,0,0,92216003,583.0,0,1,0
2,578,31,201,40.701625,-73.885304,2.0,439000,0.0,2472,2144,...,2,2,2,0,0,44413005,935.0,0,1,0
3,1310,36,199,41.817417,-71.456242,4.0,0,0.0,4968,1691,...,1,1,1,0,0,99999005,546.0,0,1,0
4,200,31,129,40.619880,-73.927260,3.0,1035000,0.0,2877,467,...,2,2,2,0,0,44531001,1121.0,0,1,0


In [90]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [91]:
rfc=RandomForestClassifier(n_estimators=100)
rfc.fit(x_train,y_train)
pred=rfc.predict(x_test)
accuracy_score(pred,y_test)

0.9932473910374463

In [92]:
from sklearn.metrics import classification_report
print(classification_report(pred,y_test))

             precision    recall  f1-score   support

          A       0.72      0.91      0.80       135
          B       0.51      0.83      0.63        92
          C       0.79      0.78      0.78       339
          D       0.93      0.87      0.90       440
          E       1.00      0.97      0.98      1864
          F       1.00      1.00      1.00      9924
          G       1.00      1.00      1.00     19786

avg / total       0.99      0.99      0.99     32580



In [93]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test,pred))

[[  123     9    21    11     7     0     0]
 [    7    76    49     9     7     0     0]
 [    4     6   264    36    25     0     0]
 [    1     1     4   384    22     0     0]
 [    0     0     1     0  1803     0     0]
 [    0     0     0     0     0  9924     0]
 [    0     0     0     0     0     0 19786]]
